In [1]:
import numpy as np
import pandas as pd
import math
import re
import seaborn as sns
import matplotlib as plt
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.model_selection import GridSearchCV
import sklearn
ridge = Ridge()
lasso = Lasso()
net   = ElasticNet()

In [2]:
train1 = pd.read_csv('./data/train_clean.csv', index_col=0)

In [3]:
#check for NA/s
train1[train1.isna().any(axis=1)]

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,TotalBath,Bedroom/Bathroom
Id,,,,,,,,,,,,,,,,,,,,,


In [4]:
trainX=train1.drop('SalePrice',axis=1)
trainY=train1['SalePrice']

In [5]:
##Ridge with 20 alphas btw 0.001 and 100. Normalizatio = True 
ridge_coefs = []
ridge_intercepts = []
ridge_R2  = []
MSE = []
alphaRange = np.linspace(1e-3,100,20)
for alpha in alphaRange:
    ridge.set_params(alpha=alpha,normalize=True)  
    ridge.fit(trainX,trainY)
    ridge_intercepts.append(ridge.intercept_)
    ridge_coefs.append(ridge.coef_)
    ridge_R2.append(ridge.score(trainX,trainY)) 
    

In [6]:
#dataframe of Ridge with 20 alphas
df_ridge_coef=pd.DataFrame(ridge_coefs,index=alphaRange,columns=trainX.columns)
df_ridge_intercept=pd.DataFrame(ridge_intercepts,index=alphaRange,columns=['Intercept'])
df_ridge_R2= pd.DataFrame(ridge_R2,index=alphaRange, columns=['R2'])
df_ridge = pd.concat([df_ridge_R2,df_ridge_intercept,df_ridge_coef], axis=1)
df_ridge.to_csv('data/ridge_v1.csv')

In [7]:
df_ridge 
#R^2 dropped below 0.5 after the 4th alpha 

,R2,Intercept,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,TotalBath,Bedroom/Bathroom
0.001000,0.913082,-1.283908e+06,49.083747,0.448824,7771.608100,5491.949726,192.833430,7.681357,22.993693,3157.620581,...,25772.260417,12104.828360,1349.869918,8670.012440,11858.814987,-1675.695866,7135.323108,-631.610683,6386.564476,5667.649882
5.264105,0.744982,-1.899022e+05,61.068412,0.140618,2772.845244,341.831940,50.416801,91.288990,12.687806,5237.036186,...,4987.894025,-2126.503802,-2520.388121,-1912.259402,661.866653,-2921.655128,-984.090333,4779.514393,2833.511680,-2099.372804
10.527211,0.642332,-9.131118e+04,47.886976,0.101460,2019.773564,63.517437,44.647941,73.606720,9.329812,3968.331697,...,3968.020255,-1991.935437,-2084.054721,-2074.188742,11.113790,-1862.936372,-983.364474,3828.894685,2187.700821,-1765.794893
15.790316,0.561739,-3.945468e+04,39.078409,0.080103,1614.537614,-15.656292,38.910257,61.506475,7.469181,3222.966038,...,3297.384438,-1749.337265,-1755.903087,-1924.165841,-150.082853,-1388.437389,-877.314817,3190.309278,1787.922822,-1498.577251
21.053421,0.497842,-5.493573e+03,32.980130,0.066373,1351.108961,-44.813505,34.197096,52.788451,6.251572,2720.969669,...,2820.627587,-1538.732582,-1512.674502,-1743.748999,-201.083934,-1113.435479,-776.776008,2732.903213,1513.727059,-1296.531233
26.316526,0.446382,1.897085e+04,28.527050,0.056737,1163.806369,-56.289312,30.403035,46.222338,5.384128,2357.027711,...,2464.225497,-1367.114649,-1327.216351,-1578.883488,-215.489158,-932.066621,-692.532341,2389.678825,1313.265781,-1140.838576
31.579632,0.404243,3.758585e+04,25.135136,0.049579,1023.059685,-60.380255,27.324040,41.103247,4.731936,2080.148500,...,2187.730178,-1227.455920,-1181.690127,-1436.338058,-215.628743,-802.742984,-623.086386,2122.811783,1160.065765,-1017.845991
36.842737,0.369192,5.228094e+04,22.465679,0.044042,913.144305,-61.092453,24.790367,37.002045,4.222587,1862.065732,...,1966.991322,-1112.534613,-1064.651480,-1314.501734,-209.864748,-705.561149,-565.528912,1909.446013,1039.068111,-918.465313
42.105842,0.339627,6.419964e+04,20.309931,0.039628,824.802814,-60.190329,22.675401,33.643383,3.813271,1685.686239,...,1786.696159,-1016.693595,-968.564965,-1210.199558,-201.726977,-629.714081,-517.311932,1734.991760,941.034484,-836.592128
47.368947,0.314378,7.407171e+04,18.532464,0.036023,752.188794,-58.517530,20.886404,30.842744,3.476898,1540.011621,...,1636.666674,-935.720587,-888.306138,-1120.371017,-192.813275,-568.794206,-476.451756,1589.710486,859.968056,-768.022090


In [8]:
##Lasso with 20 alphas btw 0.001 and 100. Normalizatio = True 
lasso_coefs = []
lasso_intercepts = []
lasso_R2  = []
alphaRange = np.linspace(1e-3,100,20)
for alpha in alphaRange:
    lasso.set_params(alpha=alpha,normalize=True)  
    lasso.fit(trainX,trainY)
    lasso_intercepts.append(lasso.intercept_)
    lasso_coefs.append(lasso.coef_)
    lasso_R2.append(lasso.score(trainX,trainY)) 
    

In [9]:
df_lasso_coef=pd.DataFrame(lasso_coefs,index=alphaRange,columns=trainX.columns)
df_lasso_intercept=pd.DataFrame(lasso_intercepts,index=alphaRange,columns=['Intercept'])
df_lasso_R2= pd.DataFrame(lasso_R2,index=alphaRange, columns=['R2'])
df_lasso = pd.concat([df_lasso_R2,df_lasso_intercept,df_lasso_coef], axis=1)
df_lasso
df_lasso.to_csv('data/lasso_v1.csv')

In [10]:
df_lasso

,R2,Intercept,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,TotalBath,Bedroom/Bathroom
0.001000,0.913336,-1.475205e+06,62.573534,0.451334,7640.022964,5614.363305,201.877654,6.949293,23.216533,3106.082646,...,27712.813811,12501.630962,1469.164953,8055.896929,11642.045335,-1585.689963,7169.136392,-2210.113093,6246.654159,5839.672722
5.264105,0.906422,-9.205038e+05,11.795347,0.344901,8724.784900,5047.185336,184.411349,0.000000,21.916323,4427.530922,...,21600.580173,4802.349302,-0.000000,5862.147680,8192.793506,-2880.000616,5054.019212,0.000000,3384.040308,797.377616
10.527211,0.893929,-5.772833e+05,-0.000000,0.305169,9417.119514,4146.839503,133.287295,0.000000,19.532774,5647.033435,...,17894.903591,0.000000,-0.000000,0.000000,1800.380680,-3787.970742,2308.067464,0.000000,3808.471855,0.000000
15.790316,0.881705,-3.720093e+05,-0.000000,0.289618,9903.308020,3639.508401,104.738457,0.000000,17.213346,6330.777006,...,15422.837564,0.000000,-0.000000,0.000000,0.000000,-3697.454859,379.206939,0.000000,4442.976634,-0.000000
21.053421,0.877612,-3.870886e+05,0.000000,0.276658,10168.892164,3266.744006,109.304902,10.761716,15.971816,7339.186955,...,15246.714282,0.000000,-0.000000,0.000000,0.000000,-2743.146144,0.000000,0.000000,4197.033706,-0.000000
26.316526,0.873517,-3.825873e+05,0.000000,0.258212,10377.306432,2929.307143,104.554034,20.449633,14.874130,8093.498979,...,15350.616101,0.000000,-0.000000,0.000000,0.000000,-1409.817976,0.000000,0.000000,4123.842092,-0.000000
31.579632,0.869619,-3.546265e+05,0.000000,0.248279,10576.441760,2591.810298,87.575893,28.161834,13.366408,8666.183180,...,15339.117987,0.000000,-0.000000,0.000000,0.000000,-184.110657,0.000000,0.000000,3995.112616,-0.000000
36.842737,0.866091,-3.230835e+05,0.000000,0.242938,10820.798589,2293.692914,71.986788,31.053574,12.505675,8992.335171,...,15189.398069,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,3924.605550,-0.000000
42.105842,0.862473,-2.924565e+05,0.000000,0.236741,11084.454557,2026.835773,59.459218,31.510659,12.133817,9265.333438,...,14966.433205,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,3878.817513,-0.000000
47.368947,0.858867,-2.696090e+05,0.000000,0.231988,11348.573639,1775.689726,50.890542,30.955849,11.819932,9493.042615,...,14727.176838,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,3835.830171,-0.000000


In [11]:

a=df_lasso.iloc[12:13,:]!=0
a[a.columns[(a != 0).any()]].columns

Index(['R2', 'Intercept', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'BsmtQual', 'BsmtFinType1',
       'BsmtFinSF1', 'TotalBsmtSF', 'HeatingQC', '1stFlrSF', 'GrLivArea',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'ScreenPorch', 'MSZoning_RM', 'LotConfig_CulDSac',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_NoRidge',
       'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_StoneBr',
       'Condition1_Feedr', 'BldgType_Twnhs', 'BldgType_TwnhsE',
       'RoofStyle_Hip', 'RoofMatl_WdShngl', 'Exterior1st_BrkFace',
       'Exterior2nd_Stucco', 'BsmtExposure_Gd', 'BsmtExposure_NoExposure',
       'SaleType_New', 'TotalBath'],
      dtype='object')


continuous and ordinal 
- some features dropped to zero with relatively small penalization
1. LotFrontage 
2. YearReModAdd - interesting it dropped to zero then back 
3. ExterCond - could be correlated to ExterQual 
4. BsmtCond - could be correlated to BsmtQual 
5. BsmtFinSF2 - could be correleated to Bsmt features
6. BsmtUnfSF - could be correlated to other Bsmt features 
7. BedrooAbvG - zero after 7 alphas 
8. GarageYrBlt - could be correlated to YrHouseBuilt 
9. GarageQual - could be correlated to OverallQual or other Garage features 
10. GarageCond - could be correlated to GarageQual or OverallQual
11. PavedDrive - could be related to OverallQual 
12. OpenPorchSF - maybe we should remove 
13. EnclosedPorchSF - maybe we should remove 
14. 3SsnPorch - zero at 6th alpha
15. PoolArea - maybe w should remove 
16. PoolQC - 
18. Fence
19. MoSold - 
20. YrSold - 
21. Gar2  (misc feature) - maybe we should remove
22. Shed (misc feature) - maybe we should remove 
23. TenC (misc feature tennis court) - maybe we should remoe 

- coefs decreased  to zero then above zero 
1. TotalBsmtSF
2. 1stFlrSf
3. ExterQual
4. TotalBsmtSF 



- coefs that increased then decreased 
1. BsmtFinType1

- coefs with increasing coefs 
1. OverallQual
2. ExterQual 
3. KitchenQual 
4.  GarageCars

- features that seem to be important
1. LotArea
2. OverallQual
3. OverallCond
4. YearBuilt
5. YearRemodAdd
6. MasVnrArea
7. ExterQual
8. BsmtQual
9. BsmtFinType1
10. BsmtFinTypeSF - maybe we should add together the SF?
11. HeatingQC
12. GrLivArea
13. KitchenAbvGr
14. KitchenQual
15. GarageFinish
16. GarageCars
17. WooddeckSF
18. TotalBaths



#CATEGORICAL 
1. MS_Zoning - seems like RM (residential medium density has the most weight on coefs)

2. Alley ( no vs paved)
3. LotShape
   - reg goes to zero quickly
4. LotConfig 
   -FR2, FR3, Inside go down to zero quickly 
5. Landslope - call go down to zero quickly 
6. Neighborhood 
  - BrDale, ClearCr, CollgCr, Gilbert, IDOTRR, MeadowV, NAmes, NPkVills, SWISU, Sawyer, Timber go down quickly 
7. condition 
  -PoSA, RRan, RRNe, RRNn go down quickly 
8. Bldgtype 
   - 2fmCon goes down to zero quicly 
9. HouseStyle
   0 2.5Unf, SFoyer, Slvl go down quickly 
10. RoofStyle
   - Gambrel,Mansard,Shed
   - Hip goes than to zero than back up 
11. RoofMatl  
   - Metal,Roll,Tar&Grv,WdShake go down quickly 
    - WdShngl seem to matter E
12. Exterior 1
    -only CemntBd,HdBoard, ImStuc, and Wd Sdng did NOT go down to zero quickly 
13. Exterior 2 
    - only _ImStucc, Stucco,Wd Shng did NOT go down to zero quickly 
     - VinylSd went to zero then back up 
14. MasVnrType
   - Seems like BrkFace is the only that did not go down to zero quicly 
15. Foundation - does not like seem any of the dummified features are important 
16. BsmtExposure
    -  only Gd and NoExposure seem to matter 
17. BsmtFinType2 - does not seem to matter 
18. Heating - only OthW seems to matter 
19. Electrical - does not seem to matter 
20. Central Air - does not seem to matter 
21. GarageType 
    -Builin went down to zero than back up 
22. SaleType 
   - only New Type seems to matter 
23. Sale Condition
    - only Family seem to matter 

In [12]:
#count of zero coefs per alpha 
df_lasso[df_lasso == 0].count(axis=1)

0.001000        0
5.264105       53
10.527211      84
15.790316     107
21.053421     123
26.316526     130
31.579632     134
36.842737     143
42.105842     145
47.368947     150
52.632053     152
57.895158     152
63.158263     156
68.421368     158
73.684474     162
78.947579     165
84.210684     165
89.473789     165
94.736895     167
100.000000    167
dtype: int64

In [145]:
from sklearn.linear_model import RidgeCV
alpha = np.linspace(1e-3,100,20)
regressor = RidgeCV(alphas=list(alpha),store_cv_values=True,normalize=True)
regressor.fit(trainX, trainY)
cv_mse = np.mean(regressor.cv_values_, axis=0)

print("Alphas: %s" % alpha)
print("Ridge RMSE on Training set :", cv_mse)
print("Best Alpha using built-in RidgeCV: %f" % regressor.alpha_)
#this is the second alpha

Alphas: [1.00000000e-03 5.26410526e+00 1.05272105e+01 1.57903158e+01
 2.10534211e+01 2.63165263e+01 3.15796316e+01 3.68427368e+01
 4.21058421e+01 4.73689474e+01 5.26320526e+01 5.78951579e+01
 6.31582632e+01 6.84213684e+01 7.36844737e+01 7.89475789e+01
 8.42106842e+01 8.94737895e+01 9.47368947e+01 1.00000000e+02]
Ridge RMSE on Training set : [1.91624139e+09 1.71344485e+09 2.32840327e+09 2.82155408e+09
 3.21474806e+09 3.53210148e+09 3.79224926e+09 4.00875372e+09
 4.19143321e+09 4.34747024e+09 4.48219961e+09 4.59964815e+09
 4.70290340e+09 4.79436769e+09 4.87593561e+09 4.94911993e+09
 5.01514231e+09 5.07499950e+09 5.12951255e+09 5.17936359e+09]
Best Alpha using built-in RidgeCV: 5.264105


In [193]:
from sklearn.linear_model import LassoCV
alpha = np.linspace(1e-3,100,20)
regressor = LassoCV(alphas=list(alpha),max_iter=10000,cv=10, normalize=True, random_state=0)
regressor.fit(trainX, trainY)
lasso_mse = np.mean(regressor.mse_path_, axis=0)
print("Alphas: %s" % alpha)
print("Lasso RMSE on Training set :", lasso_mse)
print("Best Alpha using built-in LassoCV: %f" % regressor.alpha_)

Alphas: [1.00000000e-03 5.26410526e+00 1.05272105e+01 1.57903158e+01
 2.10534211e+01 2.63165263e+01 3.15796316e+01 3.68427368e+01
 4.21058421e+01 4.73689474e+01 5.26320526e+01 5.78951579e+01
 6.31582632e+01 6.84213684e+01 7.36844737e+01 7.89475789e+01
 8.42106842e+01 8.94737895e+01 9.47368947e+01 1.00000000e+02]
Lasso RMSE on Training set : [7.96094796e+08 9.71178508e+08 6.92258988e+08 1.57743109e+09
 1.36295741e+09 7.89902615e+08 7.57982335e+08 7.83757838e+08
 3.30713021e+09 5.90120884e+08]
Best Alpha using built-in LassoCV: 21.053421


In [206]:
# select features that do not have a zero after 12 alpahs, as at alpha 62
a=df_lasso.iloc[12:13,2:]!=0
new_v1=list(a[a.columns[(a != 0).any()]].columns)
len(new_v1)
trainX_v1=trainX[new_v1]

44

In [221]:
#Try Lasso again with reduced features 
alpha = np.linspace(1e-3,100,20)
regressor = LassoCV(alphas=list(alpha),max_iter=10000,cv=10, normalize=True, random_state=0)
regressor.fit(trainX_v1, trainY)
lasso_mse = np.mean(regressor.mse_path_, axis=0)
print("Alphas: %s" % alpha)
print("Lasso RMSE on Training set :", lasso_mse)
print("Best Alpha using built-in LassoCV: %f" % regressor.alpha_)
print(new_v1)

Alphas: [1.00000000e-03 5.26410526e+00 1.05272105e+01 1.57903158e+01
 2.10534211e+01 2.63165263e+01 3.15796316e+01 3.68427368e+01
 4.21058421e+01 4.73689474e+01 5.26320526e+01 5.78951579e+01
 6.31582632e+01 6.84213684e+01 7.36844737e+01 7.89475789e+01
 8.42106842e+01 8.94737895e+01 9.47368947e+01 1.00000000e+02]
Lasso RMSE on Training set : [5.30238757e+08 6.15894452e+08 6.72057016e+08 1.50227253e+09
 1.39010390e+09 7.94636018e+08 7.70701028e+08 7.72598100e+08
 3.17863908e+09 5.81141164e+08]
Best Alpha using built-in LassoCV: 5.264105
['LotArea', 'OverallQual', 'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'BsmtQual', 'BsmtFinType1', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'KitchenAbvGr', 'KitchenQual', 'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'MSZoning_RM', 'LotConfig_CulDSac', 'Neighborhood_Crawfor', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_StoneBr', 'BldgType_Twnhs', 'BldgType_TwnhsE', 'RoofStyle_Hip', 'Ro

In [215]:
# select features based on the largest alpha
a=df_lasso.iloc[19:,2:]!=0
new_v2=list(a[a.columns[(a != 0).any()]].columns)
print(len(new_v2))
trainX_v2=trainX[new_v2]

33


In [220]:
#Try Lasso again with only 33 features from new_v2. best alpha at 5.264 for both v1 and v2 
alpha = np.linspace(1e-3,100,20)
regressor = LassoCV(alphas=list(alpha),max_iter=10000,cv=10, normalize=True, random_state=0)
regressor.fit(trainX_v2, trainY)
lasso_mse = np.mean(regressor.mse_path_, axis=0)
print("Alphas: %s" % alpha)
print("Lasso RMSE on Training set :", lasso_mse)
print("Best Alpha using built-in LassoCV: %f" % regressor.alpha_)
print(new_v2)

Alphas: [1.00000000e-03 5.26410526e+00 1.05272105e+01 1.57903158e+01
 2.10534211e+01 2.63165263e+01 3.15796316e+01 3.68427368e+01
 4.21058421e+01 4.73689474e+01 5.26320526e+01 5.78951579e+01
 6.31582632e+01 6.84213684e+01 7.36844737e+01 7.89475789e+01
 8.42106842e+01 8.94737895e+01 9.47368947e+01 1.00000000e+02]
Lasso RMSE on Training set : [5.46316917e+08 6.36127230e+08 6.96075259e+08 1.53779758e+09
 1.41180833e+09 8.21099015e+08 7.79996380e+08 7.86233244e+08
 3.20752764e+09 5.91891648e+08]
Best Alpha using built-in LassoCV: 5.264105
['LotArea', 'OverallQual', 'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'BsmtQual', 'BsmtFinType1', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'KitchenAbvGr', 'KitchenQual', 'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'MSZoning_RM', 'LotConfig_CulDSac', 'Neighborhood_Crawfor', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_StoneBr', 'BldgType_Twnhs', 'BldgType_TwnhsE', 'RoofStyle_Hip', 'Ro

In [223]:
import statsmodels.api as sm
x = sm.add_constant(trainX_v2)
model = sm.OLS(trainY, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.857
Model:                            OLS   Adj. R-squared:                  0.854
Method:                 Least Squares   F-statistic:                     257.4
Date:                Sun, 10 Nov 2019   Prob (F-statistic):               0.00
Time:                        17:13:26   Log-Likelihood:                -17028.
No. Observations:                1452   AIC:                         3.412e+04
Df Residuals:                    1418   BIC:                         3.430e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                   -3

In [224]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif_(X):
    thresh = 5.0
    variables = range(X.shape[1])

    for i in np.arange(0, len(variables)):
        vif = [variance_inflation_factor(X[variables].values, ix) for ix in range(X[variables].shape[1])]
        print(vif)
        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X[variables].columns[maxloc] + '\' at index: ' + str(maxloc))
            del variables[maxloc]

    print('Remaining variables:')
    print(X.columns[variables])
    return X

In [243]:
import pandas as pd
from sklearn.linear_model import LinearRegression

def sklearn_vif(exogs, data):

    # initialize dictionaries
    vif_dict, tolerance_dict = {}, {}

    # form input data for each exogenous variable
    for exog in exogs:
        not_exog = [i for i in exogs if i != exog]
        X, y = data[not_exog], data[exog]

        # extract r-squared from the fit
        r_squared = LinearRegression().fit(X, y).score(X, y)

        # calculate VIF
        vif = 1/(1 - r_squared)
        vif_dict[exog] = vif

        # calculate tolerance
        tolerance = 1 - r_squared
        tolerance_dict[exog] = tolerance

    # return VIF DataFrame
    df_vif = pd.DataFrame({'VIF': vif_dict, 'Tolerance': tolerance_dict})

    return df_vif

In [246]:
exogs=new_v2
data=trainX_v3
sklearn_vif(exogs=exogs,data=data)

,VIF,Tolerance
1stFlrSF,5.354458,0.186760
BldgType_Twnhs,1.184772,0.844044
BldgType_TwnhsE,1.305812,0.765807
BsmtExposure_Gd,1.455073,0.687251
BsmtExposure_NoExposure,1.419434,0.704506
BsmtFinSF1,2.950536,0.338921
BsmtFinType1,2.443972,0.409170
BsmtQual,3.031324,0.329889
ExterQual,3.007253,0.332529
FireplaceQu,4.910716,0.203636


In [247]:
exogs=list(train1.columns)
data=train1
sklearn_vif(exogs=exogs,data=data)

/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars
/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars
/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars
/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars
/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars
/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars
/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars
/Users/zhuwang/anaco

,VIF,Tolerance
1stFlrSF,inf,0.000000
2ndFlrSF,inf,0.000000
3SsnPorch,1.192172,0.838805
Alley_No,2.796133,0.357637
Alley_Pave,2.673254,0.374076
Bedroom/Bathroom,7.856675,0.127280
BedroomAbvGr,5.836141,0.171346
BldgType_2fmCon,1.949689,0.512902
BldgType_Duplex,3.857423,0.259240
BldgType_Twnhs,2.596724,0.385101
